In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from copy import copy
from multiprocessing import Pool, Process, Value
import time
from threading import Thread

In [2]:
filename = 'CRTRAITS.TXT'

data = pd.read_csv(filename, sep=',', encoding='utf-8')
data.head()

,Singular,Plural,Wood,Mercury,Ore,Sulfur,Crystal,Gems,Gold,Fight Value,...,Attack,Defense,MinDmg,MaxDmg,Shots,Spells,GuardsLow,GuardsHigh,Ability Text,Attributes
0,Pikinier,Pikinierzy,0,0,0,0,0,0,60,100,...,4,5,1,3,0,0,20,50,Powstrzymuje szarżę.,cusNO_JOUSTING
1,Halabardnik,Halabardnicy,0,0,0,0,0,0,75,115,...,6,5,2,3,0,0,20,30,Powstrzymuje szarżę.,cusNO_JOUSTING
2,Łucznik,Łucznicy,0,0,0,0,0,0,100,115,...,6,3,2,3,12,0,16,30,NaN,SHOOTING_ARMY
3,Kusznik,Kusznicy,0,0,0,0,0,0,150,115,...,6,3,2,3,24,0,16,25,Strzela dwukrotnie.,const_two_attacks | SHOOTING_ARMY
4,Gryf,Gryfy,0,0,0,0,0,0,200,324,...,8,8,3,6,0,0,12,25,Dwukrotnie kontratakuje.,DOUBLE_WIDE | FLYING_ARMY


In [3]:
data.Attributes.values[data.Attributes.values == u'0'] = ''
set(' | '.join(data.Attributes.values).split(' | '))

{u'',
 u'CATAPULT',
 u'DOUBLE_WIDE',
 u'FLYING_ARMY',
 u'HAS_EXTENDED_ATTACK',
 u'IMMUNE_TO_FIRE_SPELLS',
 u'IMMUNE_TO_MIND_SPELLS',
 u'IS_UNDEAD',
 u'KING_1',
 u'KING_2',
 u'KING_3',
 u'MULTI_HEADED',
 u'SHOOTING_ARMY',
 u'SIEGE_WEAPON',
 u'const_free_attack',
 u'const_jousting',
 u'const_lowers_morale',
 u'const_no_melee_penalty',
 u'const_no_wall_penalty',
 u'const_raises_morale',
 u'const_two_attacks',
 u'cusELEMENTAL',
 u'cusGOLEM',
 u'cusNO_JOUSTING'}

In [4]:
# do wzięcia pod uwagę w symulacjach 1vs1
keywords = {'DOUBLE_WIDE', 
            'SHOOTING_ARMY', 
            'const_free_attack', 
            'const_jousting',
            'const_no_melee_penalty',
            'const_two_attacks',
            'IS_UNDEAD',
            'cusELEMENTAL',
            'cusGOLEM'}

In [5]:
name = ['Singular']
crap = ['Plural', 'Wood', 'Mercury', 'Ore', 'Sulfur', 'Crystal', 'Gems', 'Gold', 'Ability Text']
growth = ['Growth', 'Horde Growth']
abilities = ['Attributes']
spells = ['Spells']
guards_quantity = ['GuardsLow', 'GuardsHigh']

cols_to_drop = crap + growth + guards_quantity + spells
data.drop(cols_to_drop, axis=1, inplace=True)

In [12]:
haters = set()
for x in [u'Anioł', u'Archanioł']:
    for y in [u'Diabeł', u'Arcydiabeł']:
        haters.add((x,y))
        haters.add((y,x))
for x in [u'Dżinn', u'Wielki dżinn']:
    for y in [u'Ifryt', u'Sułtański ifryt']:
        haters.add((x,y))
        haters.add((y,x))
for x in [u'Gigant', u'Tytan']:
    haters.add((u'Czarny smok', x))
haters.add((u'Tytan', u'Czarny smok'))

In [31]:
elementals = set()
pref = u'Żywiołak '
for x in [u'powietrza', u'burzy']:
    for y in [u'ziemi', u'magmy']:
        elementals.add((pref + x, pref + y))
        elementals.add((pref + y, pref + x))
for x in [u'wody', u'lodu']:
    for y in [u'ognia', u'energii']:
        elementals.add((pref + x, pref + y))
        elementals.add((pref + y, pref + x))

In [6]:
data.head()

,Singular,Fight Value,AI Value,Hit Points,Speed,Attack,Defense,MinDmg,MaxDmg,Shots,Attributes
0,Pikinier,100,80,10,4,4,5,1,3,0,cusNO_JOUSTING
1,Halabardnik,115,115,10,5,6,5,2,3,0,cusNO_JOUSTING
2,Łucznik,115,126,10,4,6,3,2,3,12,SHOOTING_ARMY
3,Kusznik,115,184,10,6,6,3,2,3,24,const_two_attacks | SHOOTING_ARMY
4,Gryf,324,351,25,6,8,8,3,6,0,DOUBLE_WIDE | FLYING_ARMY


In [96]:
class unit_type(object):
    def __init__(self, name, fightv, aiv, hp, spd, att, df, dmlow, dmhi, shots, abi):
        self.name = name
        self.fight_value = fightv
        self.ai_value = aiv
        self.hp = hp
        self.speed = spd
        self.attack = att
        self.defense = df
        self.dmg_min = dmlow
        self.dmg_max = dmhi
        self.shots = shots
        self.attributes = {x for x in abi.split(' | ') if x in keywords}
        self.hates = [v for (k,v) in haters if k == self.name]
        self.opp_elem = [v for (k,v) in elementals if k == self.name]
        
class stack(object):
    def __init__(self, unit, count):
        self.count = count
        self.name = unit.name
        
        self.hp = unit.hp
        self.hp_left = self.hp
        
        self.speed = unit.speed
        self.attack = unit.attack
        self.defense = unit.defense
        self.dmg_min = unit.dmg_min
        self.dmg_max = unit.dmg_max
        self.shots = unit.shots
        self.attributes = unit.attributes      
        self.fight_value = unit.fight_value
        self.ai_value = unit.ai_value
        self.hates = unit.hates
        self.opp_elem = unit.opp_elem
        
        self.cap = self.count
        self.rebirth_available = self.name == u'Feniks' or False
        
    def take_dmg(self, dmg):
        if dmg < self.hp_left:
            self.hp_left -= dmg
        else:
            dmg -= self.hp_left
            num_killed, rem = divmod(dmg, self.hp)
            self.count -= num_killed + 1
            self.hp_left = self.hp - rem
            self.count = max(self.count, 0)
        if self.name == u'Feniks' and not self.is_alive() and self.rebirth_available:
            self.rebirth()
        elif self.name in [u'Zjawa', u'Upiór', u'Troll']:
            self.hp_left = self.hp
            
            
    def __calc_base_damage(self, other):
        if self.count < 10:
            base_dmg = sum(np.random.randint(self.dmg_min, self.dmg_max+1, 
                                             size=self.count))
        else:
            base_dmg = sum(np.random.randint(self.dmg_min, self.dmg_max+1, 
                                             size=10)) * self.count / 10
        
        defense = other.defense
        if self.name == u'Behemot':
            defense = int(.6 * defense)
        elif self.name == u'Starożytny behemot':
            defense = int(.2 * defense)
        
        att_to_def = self.attack - defense
        base_dmg_reduction = 0.
        base_dmg_bonus = 0.
        if att_to_def > 0:
            base_dmg_bonus = min(.05 * att_to_def, 3.)
        else:
            base_dmg_reduction = min(.025 * -att_to_def, .7)
            
        return base_dmg, base_dmg_bonus, base_dmg_reduction
    
            
    def attack_melee(self, other, dmg_bonus=0., melee_penalty=False):
        base_dmg, base_dmg_bonus, base_dmg_reduction = self.__calc_base_damage(other)
        dmg_bonus += base_dmg_bonus
        dmg_reductions = [base_dmg_reduction]
        
        if melee_penalty:
            dmg_reductions.append(.5)
            
        if self.name == u'Upiorny rycerz' and np.random.rand() < .2:
            dmg_bonus += 1.
        elif other.name in self.hates:
            dmg_bonus += .5
        elif other.name in self.opp_elem:
            dmg_bonus += 1.
            
        if self.name == u'Żywiołak magii' and other.name in [u'Żywiołak magii', u'Czarny smok']:
            dmg_reductions.append(.5)
        elif self.name == u'Żywiołak umysłu' and (other.is_nonliving() or other.name in \
                                                  [u'Gigant', u'Tytan', u'Czarny smok']):
            dmg_reductions.append(.5)

        damage = base_dmg * (1. + dmg_bonus)
        for reduction in dmg_reductions:
            damage *= 1. - reduction
        
        damage = int(damage)
        other.take_dmg(damage)
        
        if self.name == u'Wampirzy lord' and not other.is_nonliving():
            self.drain_life(damage)
        elif self.name == u'Wielka gorgona' and not other.is_nonliving():
            self.death_stare(other)
        
        
    def attack_range(self, other, dmg_bonus=0., range_penalty=False):
        assert self.is_shooter() and self.shots > 0
        base_dmg, base_dmg_bonus, base_dmg_reduction = self.__calc_base_damage(other)
        dmg_bonus += base_dmg_bonus
            
        damage = base_dmg * (1. + dmg_bonus) * (1. - base_dmg_reduction)
        if range_penalty:
            damage /= 2.
            
        other.take_dmg(int(damage))
        self.shots -= 1
        
    def drain_life(self, damage_dealt):
        assert self.name == u'Wampirzy lord'
        if self.hp_left + damage_dealt <= self.hp:
            self.hp_left += damage_dealt
        else:
            damage_dealt -= self.hp - self.hp_left
            self.hp_left = self.hp
            res, rem = divmod(damage_dealt, self.hp)
            self.count = min(self.cap, self.count + res)
            if self.count < self.cap and rem:
                self.count += 1
                self.hp_left = rem
                
    
                
    def death_stare(self, other):
        assert self.name == u'Wielka gorgona' and not other.is_nonliving()
        to_death_stare = min(np.random.binomial(n=self.count, p=.1), (self.count + 9) / 10)
        other.count = max(0, other.count - to_death_stare)
        
    def rebirth(self):
        assert self.name == u'Feniks' and not self.is_alive() and self.rebirth_available
        certain, rem = divmod(self.count, 5)
        to_rebirth = certain + (np.random.rand() < .2*rem)
        self.count = to_rebirth
        self.rebirth_available = False
        
    def is_alive(self):
        return self.count > 0
        
    def is_shooter(self):
        return 'SHOOTING_ARMY' in self.attributes
    
    def strikes_twice(self):
        return 'const_two_attacks' in self.attributes and not self.is_shooter()
    
    def shoots_twice(self):
        return 'const_two_attacks' in self.attributes and self.is_shooter()
    
    def is_big(self):
        return 'DOUBLE_WIDE' in self.attributes
    
    def melee_penalty(self):
        return self.is_shooter() and not 'const_no_melee_penalty' in self.attributes
    
    def no_retaliation(self):
        return 'const_free_attack' in self.attributes
    
    def range_penalty(self):
        return self.is_shooter() and self.name != 'Strzelec'
    
    def is_elemental(self):
        return 'cusELEMENTAL' in self.attributes
    
    def is_undead(self):
        return 'IS_UNDEAD' in self.attributes
    
    def is_golem(self):
        return 'cusGOLEM' in self.attributes
    
    def is_nonliving(self):
        return self.is_elemental() or self.is_undead() or self.is_golem()

In [97]:
def make_unit(name):
    return unit_type(*data.values[data.Singular.values == name][0])

In [98]:
def fight(stackA, stackB, num_iter):    
    wins = {stackA.name : [0,0],
            stackB.name : [0,0]}
    
    def units_order(s1, s2):
        temp = sorted([s1,s2], key=lambda x: x.speed, reverse=True)
        if s1.speed == s2.speed and np.random.rand() < .5:
            return reversed(temp)
        return temp    
    
    def melee_hit(current, other):
        current.attack_melee(other, melee_penalty=current.melee_penalty())
        
        if other.is_alive() and not current.no_retaliation():
            other.attack_melee(current, melee_penalty=other.melee_penalty())
            
        if current.is_alive() and current.strikes_twice():
            current.attack_melee(other)
        return other, current
    
    def range_hit(current, other, apply_penalty):
        penalty = current.range_penalty() if apply_penalty else False
        current.attack_range(other, range_penalty=penalty)
        if current.shoots_twice() and current.shots > 0:
            current.attack_range(other, range_penalty=penalty)
        return other, current
    
    def fight_to_death(current, other):
        while current.is_alive() and other.is_alive():
            if other.name == u'Błękitny smok' and not current.is_nonliving() and np.random.rand() < .1:
                current, other = other, current
            else:
                current, other = melee_hit(current, other)
        return current, other
    
    def walker_vs_shooter(walker, shooter):            
        to_walk = starting_dist - 1
        first_move = to_walk % walker.speed
        if first_move == 0:
            first_move = walker.speed
        avoid_by_move = to_walk - first_move > 10
        avoid_by_wait = False
            
        if walker.speed < shooter.speed:
            num_shots = to_walk / walker.speed + (to_walk % walker.speed > 0)
        elif walker.speed > shooter.speed:
            num_shots = to_walk / walker.speed - (to_walk % walker.speed == 0)
            avoid_by_wait = True
        else:
            if np.random.rand() < .5:
                num_shots = to_walk / walker.speed + (to_walk % walker.speed > 0)
            else:
                num_shots = to_walk / walker.speed - (to_walk % walker.speed == 0)
                    
        num_full_shots = max(0, num_shots - avoid_by_move - avoid_by_wait)
        num_half_shots = num_shots - num_full_shots
            
        for j in xrange(num_half_shots):
            range_hit(shooter, walker, apply_penalty=True)
        for j in xrange(num_full_shots):
            range_hit(shooter, walker, apply_penalty=False)
            
        current, other = walker, shooter
        return fight_to_death(current, other)
    
    starting_dist = 14
    if stackA.is_big():
        starting_dist -= 1
    if stackB.is_big():
        starting_dist -= 1
        

    for it in xrange(num_iter):
        current, other = units_order(copy(stackA), copy(stackB))
        
        if not current.is_shooter() and not other.is_shooter():
            current, other = fight_to_death(current, other)
                
        elif current.is_shooter() and other.is_shooter():
            full_round = True
            while current.is_alive() and ((current.shots > 0 and other.shots > 0) or not full_round):
                current, other = range_hit(current, other, apply_penalty=True)
                full_round = not full_round
            if current.is_alive():
                if current.shots == 0 and other.shots == 0:
                    current, other = units_order(current, other)
                    current, other = fight_to_death(current, other)
                else:
                    shooter = current if current.shots > 0 else other
                    walker = current if current.shots == 0 else other
                    current, other = walker_vs_shooter(walker=walker, shooter=shooter)
                
        else:           
            shooter = current if current.is_shooter() else other
            walker = current if not current.is_shooter() else other
            current, other = walker_vs_shooter(walker=walker, shooter=shooter)
                
            
        winner = current if current.is_alive() else other
        wins[winner.name][0] += 1
        wins[winner.name][1] += winner.count
        
    for s in [stackA.name, stackB.name]:
        if wins[s][0]:
            wins[s][1] /= float(wins[s][0])
    
    return wins

In [99]:
def find_balance(nameA, nameB, num_iter):
    
    def balanced(result):
        return (num_iter / 2) * .95 < result[A.name][0] < (num_iter / 2) * 1.05
    
    unitA = make_unit(nameA)
    unitB = make_unit(nameB)
    A = stack(unitA, unitB.ai_value * 10)
    B = stack(unitB, unitA.ai_value * 10)
    
    res = fight(A, B, num_iter)
    
    if balanced(res):
        return A.count, B.count
    
    last_loser = A if res[A.name][0] < res[B.name][0] else B
    inc = last_loser.count / 10
    enough = False
    
    while not enough:
        low = last_loser.count
        last_loser.count += inc
        res = fight(A, B, num_iter)
        loser = A if res[A.name][0] < res[B.name][0] else B
        enough = last_loser is not loser
        if not enough:
            last_loser = loser
        
    if balanced(res):
        return A.count, B.count
    
    high = last_loser.count
    
    # here binsearch from (low, high) for balanced result
    X = last_loser
    
    while True:
        middle = low + (high - low) / 2
        X.count = middle
        res = fight(A, B, num_iter)
        if balanced(res) or np.abs(high - low) <= 1:
            return A.count, B.count
        loser = A if res[A.name][0] < res[B.name][0] else B
        if X is loser:
            low = middle + 1
        else:
            high = middle - 1            

In [100]:
t0 = time.time()
print find_balance(u'Troll', u'Ogr', 500)
time.time() - t0

(8033, 10240)


1.215467929840088

In [15]:
t0 = time.time()
A = stack(make_unit(u'Archanioł'), 1)
B = stack(make_unit(u'Starożytny behemot'), 1)
B.speed = 20

result = fight(A, B, 1000)
print result
time.time()-t0

{u'Staro\u017cytny behemot': [0, 0], u'Archanio\u0142': [1000, 1.0]}


0.13994908332824707

In [13]:
castle_units = data.values[:14]
all_units = data.values[:-5]
castle_scores = np.zeros((14, 141))

t0 = time.time()
x_id = 0
i = 0
for x in castle_units:
    u_id = 0
    for u in all_units:
        if x[0] == u[0]:
            castle_scores[x_id, u_id] = 1.
        else:
            cnt_x, cnt_u = find_balance(x[0], u[0], 500)
            castle_scores[x_id, u_id] = cnt_u / float(cnt_x)
#             print x[0], u[0], castle_scores[x_id, u_id]
            i += 1
            if not i % 20:
                print 'Done {} pairs in {:.2f}s'.format(i, time.time()-t0)
        u_id += 1
    x_id += 1

Done 20 pairs in 36.86s
Done 40 pairs in 69.91s
Done 60 pairs in 103.15s
Done 80 pairs in 136.31s
Done 100 pairs in 165.20s
Done 120 pairs in 203.26s
Done 140 pairs in 260.18s
Done 160 pairs in 295.62s
Done 180 pairs in 326.15s
Done 200 pairs in 355.46s
Done 220 pairs in 378.72s
Done 240 pairs in 406.92s
Done 260 pairs in 434.56s
Done 280 pairs in 487.37s
Done 300 pairs in 527.85s
Done 320 pairs in 562.67s
Done 340 pairs in 604.99s
Done 360 pairs in 643.26s
Done 380 pairs in 682.33s
Done 400 pairs in 725.80s
Done 420 pairs in 789.31s
Done 440 pairs in 830.55s
Done 460 pairs in 882.27s
Done 480 pairs in 923.03s
Done 500 pairs in 963.29s
Done 520 pairs in 1006.40s
Done 540 pairs in 1049.69s
Done 560 pairs in 1100.50s
Done 580 pairs in 1132.52s
Done 600 pairs in 1171.61s
Done 620 pairs in 1207.68s
Done 640 pairs in 1238.75s
Done 660 pairs in 1274.76s
Done 680 pairs in 1307.09s
Done 700 pairs in 1357.63s
Done 720 pairs in 1397.10s
Done 740 pairs in 1433.60s
Done 760 pairs in 1472.44s
Done 

In [14]:
np.savez('misc/castle_scores.npz', castle_scores)

In [15]:
a = np.load('misc/castle_scores.npz')['arr_0']
cs = pd.DataFrame.from_records(a.T, columns=data.Singular.values[:14])
cs.insert(0, 'Nazwa', pd.Series(data.Singular.values[:-5]))
cs

,Nazwa,Pikinier,Halabardnik,Łucznik,Kusznik,Gryf,Gryf królewski,Zbrojny,Krzyżowiec,Mnich,Kapłan,Kawalerzysta,Czempion,Anioł,Archanioł
0,Pikinier,1.000000,1.190476,1.575000,2.618750,2.723041,2.835443,4.630593,6.275347,8.598750,10.122500,14.742424,16.166282,39.088785,64.481999
1,Halabardnik,0.842609,1.000000,1.056161,2.243478,2.394270,2.473771,3.940870,5.496522,6.061739,8.681739,12.811060,14.075067,33.981043,54.206300
2,Łucznik,0.634921,0.946032,1.000000,1.560317,2.994444,3.087526,3.824603,5.337302,4.846825,5.091650,15.660317,17.030159,40.579365,77.703175
3,Kusznik,0.382409,0.447297,0.641548,1.000000,1.907609,2.708152,1.868963,2.972700,2.763043,3.696402,13.583696,14.658696,34.863587,77.504891
4,Gryf,0.367236,0.418519,0.333952,0.499322,1.000000,1.052879,1.657550,2.219373,1.532194,2.911396,5.494071,6.057265,14.611681,23.192389
5,Gryf królewski,0.353125,0.403348,0.323437,0.370296,0.949777,1.000000,1.598214,2.017857,1.468080,2.050670,5.120759,5.734598,14.126116,22.282589
6,Zbrojny,0.216629,0.253751,0.261465,0.535056,0.603300,0.625087,1.000000,1.366517,1.498652,2.022697,3.235783,3.566576,8.668394,13.706075
7,Krzyżowiec,0.159014,0.182482,0.187361,0.344218,0.450578,0.495575,0.731788,1.000000,0.845238,1.574320,2.612081,2.890173,7.051138,10.880238
8,Mnich,0.116296,0.158076,0.206321,0.361635,0.652538,0.681162,0.655376,1.181435,1.000000,1.192369,3.582474,3.888169,9.188942,17.580128
9,Kapłan,0.098644,0.115254,0.196400,0.270800,0.343411,0.487964,0.494444,0.635195,0.838667,1.000000,2.162222,2.827067,6.759733,12.199333


In [16]:
cs.to_csv('castle_scores/castle_scores.csv', encoding='utf-8')

In [60]:
castle_scores_ai_value_based = np.zeros((14, 141))

x_id = 0
for x in castle_units:
    u_id = 0
    for u in all_units:
        x_aiv = data['AI Value'].values[x_id]
        u_aiv = data['AI Value'].values[u_id]
        castle_scores_ai_value_based[x_id, u_id] = x_aiv / float(u_aiv)
        u_id += 1
    x_id += 1

In [61]:
np.savez('misc/castle_scores_ai_value_based.npz', castle_scores_ai_value_based)

In [18]:
b = np.load('misc/castle_scores_ai_value_based.npz')['arr_0']
csaiv = pd.DataFrame.from_records(b.T, columns=data.Singular.values[:14])
csaiv.insert(0, 'Nazwa', pd.Series(data.Singular.values[:-5]))
csaiv

,Nazwa,Pikinier,Halabardnik,Łucznik,Kusznik,Gryf,Gryf królewski,Zbrojny,Krzyżowiec,Mnich,Kapłan,Kawalerzysta,Czempion,Anioł,Archanioł
0,Pikinier,1.000000,1.437500,1.575000,2.300000,4.387500,5.600000,5.562500,7.350000,6.062500,9.375000,24.325000,26.250000,62.737500,109.700000
1,Halabardnik,0.695652,1.000000,1.095652,1.600000,3.052174,3.895652,3.869565,5.113043,4.217391,6.521739,16.921739,18.260870,43.643478,76.313043
2,Łucznik,0.634921,0.912698,1.000000,1.460317,2.785714,3.555556,3.531746,4.666667,3.849206,5.952381,15.444444,16.666667,39.833333,69.650794
3,Kusznik,0.434783,0.625000,0.684783,1.000000,1.907609,2.434783,2.418478,3.195652,2.635870,4.076087,10.576087,11.413043,27.277174,47.695652
4,Gryf,0.227920,0.327635,0.358974,0.524217,1.000000,1.276353,1.267806,1.675214,1.381766,2.136752,5.544160,5.982906,14.299145,25.002849
5,Gryf królewski,0.178571,0.256696,0.281250,0.410714,0.783482,1.000000,0.993304,1.312500,1.082589,1.674107,4.343750,4.687500,11.203125,19.589286
6,Zbrojny,0.179775,0.258427,0.283146,0.413483,0.788764,1.006742,1.000000,1.321348,1.089888,1.685393,4.373034,4.719101,11.278652,19.721348
7,Krzyżowiec,0.136054,0.195578,0.214286,0.312925,0.596939,0.761905,0.756803,1.000000,0.824830,1.275510,3.309524,3.571429,8.535714,14.925170
8,Mnich,0.164948,0.237113,0.259794,0.379381,0.723711,0.923711,0.917526,1.212371,1.000000,1.546392,4.012371,4.329897,10.348454,18.094845
9,Kapłan,0.106667,0.153333,0.168000,0.245333,0.468000,0.597333,0.593333,0.784000,0.646667,1.000000,2.594667,2.800000,6.692000,11.701333


In [65]:
csaiv.to_csv('castle_scores/castle_scores_ai_value_based.csv', encoding='utf-8')

In [19]:
comparison = cs.values[:,1:] / csaiv.values[:,1:]
comparison[np.abs(comparison-1) < .2] = 1.
comparison[comparison == 1.] = ''
comp_df = pd.DataFrame.from_records(comparison, columns=data.Singular.values[:14])
comp_df.insert(0, 'Nazwa', pd.Series(data.Singular.values[:-5]))

comp_df

,Nazwa,Pikinier,Halabardnik,Łucznik,Kusznik,Gryf,Gryf królewski,Zbrojny,Krzyżowiec,Mnich,Kapłan,Kawalerzysta,Czempion,Anioł,Archanioł
0,Pikinier,,,,,0.620636,0.506329,,,1.41835,,0.606061,0.615858,0.623053,0.587803
1,Halabardnik,1.21125,,,1.40217,0.784447,0.635008,,,1.43732,1.3312,0.757077,0.770777,0.778605,0.710315
2,Łucznik,,,,,,,,,1.25918,,,,,
3,Kusznik,,0.715675,,,,,0.772785,,,,1.28438,1.28438,1.27812,1.62499
4,Gryf,1.61125,1.27739,,,,,1.30742,1.32483,,1.36253,,,,
5,Gryf królewski,1.9775,1.5713,,,1.21225,,1.60899,1.53741,1.35608,1.22493,,1.22338,1.26091,
6,Zbrojny,1.205,,,1.29402,0.764868,0.620901,,,1.37505,1.20013,0.73994,0.755774,0.768566,0.694987
7,Krzyżowiec,,,,,0.754814,0.650442,,,,1.23427,0.789262,,,0.728986
8,Mnich,0.705044,0.666667,0.794171,,,0.737418,0.714286,,,0.771065,,,,
9,Kapłan,,0.751654,,,0.733783,,,,1.29691,,,,,


In [20]:
comp_df.to_csv('castle_scores/castle_comparison.csv', encoding='utf-8')